In [2]:
%pip install regex
%pip install nltk
%pip install unicodedata
%pip install contractions
%pip install inflect
%pip install emoji
%pip install spacy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement unicodedata (from versions: none)
ERROR: No matching distribution found for unicodedata


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
!python -m spacy download ru_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB 

In [4]:
import regex as re
import nltk
import emoji
import unicodedata
import contractions
import inflect
import spacy
import numpy as np
import os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Кирилл\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Кирилл\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Кирилл\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
!python -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB 

In [9]:
def clean_text(input_text):

    # HTML Tags: The first step is to remove all HTML tags inside the input text
    clean_text = re.sub('<[^<]+?>', '', input_text)

    # URLs and Links: Next, we remove every URL and link from the text
    clean_text = re.sub(r'http\S+', '', clean_text)

    # Emojis and Emoticons: We use the self-defined function below to convert emojis to text
    # This is important for understanding the sentiment of the text being presented
    clean_text = convert_emojis_to_words(clean_text)

    # Lowercase all the input data
    clean_text = clean_text.lower()

    # Remove all White Spaces
    # Since all the data is now words, let's clean any white spaces
    clean_text = re.sub('\s+', ' ', clean_text)

    # Accented Characters to ASCII Characters: We use the unicode normalize function to convert all accented characters to ASCII characters
    #clean_text = unicodedata.normalize('NFKD', clean_text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    # Expand contractions: Text often contains words like "don't" or "won't", let us expand those
    clean_text = contractions.fix(clean_text)

    # Remove special characters: Removing anything that is not "words"
    clean_text = re.sub('[^a-zA-ZА-Яа-я0-9\s\.]', '', clean_text)

    # Convert number words to numeric form
    #temp = inflect.engine()
    #words = []
    #for word in clean_text.split():
        #if word.isdigit():
            #words.append(temp.number_to_words(word))
        #else:
            #words.append(word)
    #clean_text = ' '.join(words)

    # Remove stopwords
    stop_words = set(stopwords.words('russian'))
    tokens = word_tokenize(clean_text)
    tokens = [token for token in tokens if token not in stop_words]
    clean_text = ' '.join(tokens)

    # Add full-stop to end of sentences
    clean_text = re.sub('([а-я])\.([А-Я])', r'\1. \2', clean_text)

    # Remove punctuations
    clean_text = re.sub(r'[^\w\s.]', '', clean_text)
    # Ваши слова
    word1 = "прессслужба"
    word2 = "cookie.я согласенподписка рассылку"

    # Регулярное выражение для удаления части строки после слова "конец"
    pattern = rf'\b{re.escape(word1)}\b[\s\S]*?\b{re.escape(word2)}\b'


    # Применяем регулярное выражение
    clean_text = re.sub(pattern, '', clean_text)
    clean_text = re.sub("поступающимстудентамвыпускникаммедиатекауниверситетобразованиенаукабизнесу2версия слабовидящих", '', clean_text)
    clean_text = re.sub("личный кабинетlms moodle", '', clean_text)


    # Return the preprocessed, clean text
    return clean_text

In [10]:
def convert_emojis_to_words(text):
    
    # Convert emojis to words
    text = emoji.demojize(text, delimiters=(" ", " "))
    
    # Remove the : from the words and replace _ with space
    text = text.replace(":", "").replace("_", " ")
    
    return text

In [11]:
def remove_noise_boilerplate(input_text, min_cluster_size=2, num_clusters=5, max_noise_ratio=0.3):
    
    # Sentence split: To identify boilerplate/noise we will first need to separate sentences to find similarity
    sentences = re.split('\. |\? |\! |\n', input_text)
    
    # Convert sentences to a matrix of word embeddings
    embeddings_matrix = text_vectorize(sentences)
    
    # KMeans Clustering: Cluster the sentences to bring similar embeddings together 
    kmeans_model = KMeans(n_clusters=num_clusters)
    kmeans_model.fit(embeddings_matrix)
    model_labels = kmeans_model.labels_
    model_centroids = kmeans_model.cluster_centers_
    
    # Individual cluster size
    cluster_sizes = np.bincount(model_labels)
    
    # Identify clusters with noise and boilerplate language
    is_noise = np.zeros(num_clusters, dtype=bool)
    for i, centroid in enumerate(model_centroids):
        if cluster_sizes[i] < min_cluster_size:
            # We should ignore clusters with fewer sentences than min_cluster_size threshold
            continue
        distances = np.linalg.norm(embeddings_matrix[model_labels == i] - centroid, axis=1)
        median_distance = np.median(distances)
        if np.count_nonzero(distances > median_distance) / cluster_sizes[i] > max_noise_ratio:
            is_noise[i] = True
    
    # Remove: Sentences that are in the noise bucket, we remove them (boilerplate)
    filtered_sentences = []
    for i, sentence in enumerate(sentences):
        if not is_noise[model_labels[i]]:
            filtered_sentences.append(sentence)
    
    # Bring the sentence together
    filtered_text = ' '.join(filtered_sentences)
    
    return filtered_text

In [12]:
def text_vectorize(input_text):
    
    # Instantiate the CountVectorizer object
    vectorizer = CountVectorizer()
    
    # Use vectorizer.fit to transform the text into a matrix of word counts
    counts_matrix = vectorizer.fit_transform(input_text)
    
    # Convert to a dense matrix
    dense_matrix = counts_matrix.todense()
    
    # Return the dense matrix as a numpy array
    return np.array(dense_matrix)

In [13]:
nlp = spacy.load("ru_core_news_sm")

In [15]:
def pos_tag(input_text):

    doc = nlp(input_text)
    tagged_output = []

    # Iterate over each token in the document
    for token in doc:
        # Append the token text and its POS tag to the tagged_output list
        tagged_output.append(token.text + '_' + token.pos_)

    # Join the tagged_output list into a single string
    tagged_output_str = ' '.join(tagged_output)

    return tagged_output_str

In [16]:
def lemmatize_and_vectorize(tagged_text):
    
    # Convert the tagged text to a string
    text = " ".join([word.split("_")[0] for word in tagged_text.split()])

    # Apply the Spacy pipeline to the text
    doc = nlp(text)

    vector_list = []

    # Iterate over each token in the Spacy document
    for token in doc:
        lemma = token.lemma_

        # Get the part-of-speech tag for the token
        pos = token.pos_

        if pos == "VERB":
            vec = token.vector
        else:
            vec = token.vector + nlp(pos).vector
        
        vector_list.append(vec)

    vector_array = np.array(vector_list)

    return vector_array

In [ ]:
def get_file_paths(directory):
    file_paths = []
    for root, directories, files in os.walk(directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            file_paths.append(file_path)
    return file_paths

In [17]:
input_text = """

Студентам | Официальный сайт НИТУ МИСИС   
  





  ПоступающимСтудентамВыпускникамМедиатекаУниверситетОбразованиеНаукаБизнесу×2Версия для слабовидящихЛичный кабинетСтудентамАдаптационный проект «Погружение»Лекториум «История. Техника. Производство»Гид первокурсника Университета МИСИССтуденческий офисСтуденческий советСтипендиальные программыСоциальная поддержка студентовКарта возможностей студентаОткрытый ректоратСтуденческие объединенияПрактики и стажировкиТрудоустройствоЦифровые сервисыОнлайн-курсыWi-Fi в университетеШтаб #МыВместе
Обучение
Единая страница расписания занятий
Расписание учебных занятий (xls)
Расписание экзаменационной сессии
Ликвидация задолженностей
Институты
Кафедры
Кураторы
LMS Moodle
Онлайн-курсы
Лаборатории и научные центры
Аспирантура

Студенческий офис
Стипендиальные программы
Социальная поддержка студентов
Оплата общежития
Платное обучение
Переход с платного обучения
Перевод в университет из других вузов
Выпускникам: получение дипломов
Психологическая служба «Точка опоры»
Оформление банковской карты

Студенческая жизнь
Студенческие объединения
Олимпиады
Центр карьеры и практической подготовки
Практики и стажировки
Профком университета
Иностранным студентам

Кампус
Институты
Общежития
Цифровые сервисы
Библиотека
Медицинский пункт
Военно-учетный стол
Портал Госуслуги
Объявления31 октября Студенческий совет общежитий НИТУ МИСИС — победитель в номинации «Наш студенческий дом»30 октября Студенты российских вузов получат поддержку своих стартапов от крупного бизнесаУниверситет МИСИС приглашает студентов и аспирантов российских вузов на DemoDay с представителями крупного бизнеса. «Газпром нефть», Splat Global, CyberPhysics, АВП «Технология», ГК «Геоскан», Cesca, Embedika и другие компании представят реальные кейсы, к реализации которых они готовы привлечь молодежные команды. Коллективы, зарегистрированные на платформе «Прайм-тайм» и предложившие наиболее оригинальные решения, получат финансирование, помощь с привлечением инвесторов и возможность трудоустройства в ведущие компании. Проект реализуется в рамках программы «Приоритет 2030» при поддержке компании «Иннопрактика».2 октября Наставники студентов-переводниковВсе новости по темеЗаявки на практики и стажировки8 ноября Cтажировки в Правительство Москвы30 октября Стажировка в Газпромбанке16 октября Практика в АО «Золото Селигдара»10 октября Практика в SERCONS3 октября Практика и стажировка в Инновационном центре «Безопасный транспорт»Студенческая жизнь
Наука в Университете МИСИС — это то, что случится с тобой неизбежно. Она будет окружать тебя все время. Студенческое научное общество станет твоим проводником в мир науки: от экскурсий в лаборатории до крупнейших в России научно-популярных событий. Наука может быть яркой! 
Студия вокала, ансамбль современного танца «Андеор», театральное пространство, вокально-инструментальные и рок-группы, КВН, хип-хоп и cover-dance команды — всё это творческие студии университета, в которых может заниматься каждый! Клуб интернациональной дружбы Университета МИСИС объединяет студентов из разных стран для их адаптации и успешного взаимодействия с однокурсниками и преподавателями. На сегодняшний день в составе КИД насчитывается 30 землячеств.
Студенческий телеканал MISIS Media рассказывает обо всех событиях университета и освещают все самые свежие новости, но и снимают собственные шоу. Второй телеканал — Science MISIS, освещает актуальные новости науки, доказывая, что она также может быть интересной, увлекательной и весёлой.
 В Университете МИСИС активно развивается туризм и спортивный клуб. Спортклуб сорганизует и поддерживает сборные команды по различным видам спорта, проводит соревнования среди студентов и сотрудников университета.
Пресс-служба

 press@edu.misis.ru
Заявка на информационную поддержку
Заявка на информационную поддержку для ученых


Нашли опечатку?
Выделите ее и нажмите Ctrl+Enter. Спасибо за участие! 
Сервис предназначен только для отправки сообщений об орфографических и пунктуационных ошибках.
Приемная комиссия:
+7 499 649-44-80
vopros@misis.ru

Абитуриенту
Центр коммерциализации технологий
+7 495 638-46-57
cctt@misis.ru

Работа в Университете МИСИС
Сведения об образовательной организации


Канцелярия:
+7 495 955-00-32 
 Факс: +7 499 236-21-05 
kancela@misis.ru
 119049, Москва, Ленинский пр-кт, д. 4, стр. 1 
					Написать намПодписаться на рассылку


Техническая поддержка сайта: 
site@edu.misis.ru
Карта сайта

Информация о закупках 
Противодействие коррупции

Бета-версия
 Вопросы и предложения 
 по сайту: site@edu.misis.ru


© 2023 Университет науки и технологий МИСИС     Написать намПодписаться на рассылкуНаш сайт использует файлы cookie.
      Мы не идентифицируем вас, а улучшаем работу сайта.
      Оставаясь, вы даете согласие на обработку файлов cookie.Я согласен×Подписка на рассылку×"""

In [21]:
clean_text = clean_text(input_text)
print(clean_text)

TypeError: 'str' object is not callable

In [33]:
# Получаем список всех файлов с расширением .txt в директории
directory_path = '../Data/htmls_and_txts/txts'
uploaded_file_names = get_file_paths(directory_path)
# Замените 'путь_к_целевой_директории' на путь к директории, куда вы хотите сохранить модифицированные файлы
target_directory = '../Data/htmls_and_txts/txts_clean'


# Цикл по всем найденным файлам
for uploaded_file_path in uploaded_file_names:
    # Считываем содержимое файла
    with open(uploaded_file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Модификация содержимого (в данном случае, не делаем никаких изменений)
    modified_content = clean_text(content)

    # Создаем новый путь для файла с добавлением "_clean" в целевой директории
    new_file_path = os.path.join(target_directory, os.path.basename(uploaded_file_path)[:-4] + "_clean.txt")

    # Записываем модифицированное содержимое в новый файл
    with open(new_file_path, 'w', encoding='utf-8') as new_file:
        new_file.write(modified_content)

    print(f"Файл {uploaded_file_path} обработан. Результат записан в {new_file_path}")